In [1]:
import findspark
findspark.init()

In [4]:
#For this task, the code found in the reference link posted in the midterm doc was used for construction of the bloomfilters
#rdds are then converted to dataframes and then joined through a broadcast join
from pyspark.sql import SparkSession
from pyspark import SparkContext
from csv import reader
import sys
spark = SparkSession.builder.appName("test").getOrCreate()

In [25]:
rdd1 = sc.textFile("C:/Users/jack_/Desktop/Big_data/drive_stats_2019_Q1/*.csv")
rdd2 = sc.textFile("C:/Users/jack_/Desktop/Big_data/drive_stats_2019_Q1/2019-03-31.csv")
rdd1 = rdd1.mapPartitions(lambda x: reader(x))
rdd2 = rdd2.mapPartitions(lambda x: reader(x))

In [8]:
#this is the code borrowed from https://www.duedil.com/blogs/efficient-broadcast-joins-in-spark-using-bloom-filters
#credit for this implementation of bloom filters goes to the writer of this article
from pybloom import BloomFilter
def _merge_bloom_filters(filter_a, filter_b):
   return filter_a.union(filter_b)

def _construct_bloom_filter(records, bloom_capacity, bloom_fpr):
   bloom_filter = BloomFilter(bloom_capacity, bloom_fpr)
   for record in records:
       bloom_filter.add(record)
   yield bloom_filter

def bloom_filter_from_rdd(rdd, bloom_fpr=0.01, bloom_capacity=None, num_partitions=30):
    if not bloom_capacity:
       bloom_capacity = rdd.count()
    if bloom_capacity == 0:
       return set() 
    bloom_filters_rdd = rdd \
       .coalesce(num_partitions) \
       .mapPartitions(lambda records: _construct_bloom_filter(records, bloom_capacity, bloom_fpr))
    return bloom_filters_rdd.reduce(_merge_bloom_filters)

In [26]:
bloomFilter = bloom_filter_from_rdd(rdd2)
bFiltered = rdd1.filter(lambda x: bloomFilter.add(x))

In [33]:
from pyspark.sql.functions import broadcast
df1 = bFiltered.toDF()
df2 = filtered2.toDF()
joined = df1.join(broadcast(df2), df1._3 == df2._3)
joined.show()

+----------+-------------+-------------+--------------+-------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+---------